In [1]:
import pandas as pd
from pybaseball import schedule_and_record

In [2]:
years = [2015, 2016, 2017, 2018, 2019]
teams = ["ATL", "BAL", "CHW", "CIN", "CLE",\
         "DET", "MIL", "MIN", "SDP", "STL", "TEX"]

In [3]:
data = pd.DataFrame()
for team in teams:
    for year in years:
        d = schedule_and_record(year, team)
        d['Year'] = str(year)
        data = data.append(d, ignore_index=True)

In [4]:
data.iloc[160:162,:]

,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled,Year
160,"Sunday, Oct 4 (1)",ATL,Home,STL,W,6.0,0.0,9.0,66-95,4.0,23.0,Miller,Lackey,None,2:14,D,NaN,2.0,None,2015
161,"Sunday, Oct 4 (2)",ATL,Home,STL,W,2.0,0.0,9.0,67-95,4.0,23.0,Wisler,Lynn,Jackson,2:19,D,31441.0,3.0,2015-10-03 (Rain),2015


In [5]:
adj = data.copy(deep=True)

In [6]:
adj['DblHdrGm#'] = adj['Date'].apply(lambda row: row[-2] if row[-3] == '(' else 0).astype(int)
adj['DblHdr'] = adj['Date'].apply(lambda row: 1 if row[-3] == '(' else 0).astype(int)
adj['Date'] = adj['Date'].apply(lambda row: row[:-4] if row[-3] == '(' else row)

In [7]:
adj.iloc[160:162,:]

,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,...,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled,Year,DblHdrGm#,DblHdr
160,"Sunday, Oct 4",ATL,Home,STL,W,6.0,0.0,9.0,66-95,4.0,...,Lackey,None,2:14,D,NaN,2.0,None,2015,1,1
161,"Sunday, Oct 4",ATL,Home,STL,W,2.0,0.0,9.0,67-95,4.0,...,Lynn,Jackson,2:19,D,31441.0,3.0,2015-10-03 (Rain),2015,2,1


In [8]:
adj['Date'] = (adj['Date'] + ' ' + (adj['Year'])).astype('datetime64')

In [9]:
adj['Year'] = adj['Date'].dt.year
adj['Month'] = adj['Date'].dt.month
adj['Week'] = adj['Date'].dt.week
adj['Day_of_Year'] = adj['Date'].dt.dayofyear
adj['Day_of_Month'] = adj['Date'].dt.day
adj['Day_of_Week'] = adj['Date'].dt.dayofweek
adj['Weekend'] = adj['Day_of_Week'].apply(lambda row: 1 if row in [5,6] else 0)

In [10]:
adj['Home_Away'] = adj.Home_Away.map({'@':0, 'Home':1})

In [11]:
adj['Walkoff'] = adj[pd.notnull(adj['W/L'])]['W/L'].apply(lambda row: 1 if len(row) > 1 else 0).astype(int)
adj['W/L'] = adj[pd.notnull(adj['W/L'])]['W/L'].apply(lambda row: 1 if row[0] == 'W' else 0).astype(int)

In [12]:
adj['Wins'] = adj[pd.notnull(adj['W-L'])]['W-L'].apply(lambda row: row.split('-')[0]).astype(int)
adj['Losses'] = adj[pd.notnull(adj['W-L'])]['W-L'].apply(lambda row: row.split('-')[1]).astype(int)
adj['W-L'] = round(adj['Wins']/(adj['Wins']+adj['Losses']),3)

In [13]:
adj['Shortened'] = adj[pd.notnull(adj['Inn'])]['Inn'].apply(lambda row: 1 if row < 9 else 0).astype(int)
adj['Extra'] = adj[pd.notnull(adj['Inn'])]['Inn'].apply(lambda row: 1 if row > 9 else 0).astype(int)

In [14]:
adj['GB'] = adj[pd.notnull(adj['GB'])]['GB'].apply(lambda row: row.split('up')[1] if row.split('up')[0] == ''
                                                   else '-' + row.split('up')[0])
adj['GB'] = adj.GB.replace({'-Tied':0}).astype(float)

In [15]:
adj['Team_Pitcher'] = adj[pd.notnull(adj['W/L'])].apply(lambda row: row['Win'] if row['W/L'] == 1 else row['Loss'], axis=1)
adj['Opp_Pitcher'] = adj[pd.notnull(adj['W/L'])].apply(lambda row: row['Win'] if row['W/L'] == 0 else row['Loss'], axis=1)

In [16]:
adj['Duration'] = adj[pd.notnull(adj['Time'])]['Time'].str.split(':').apply(lambda row: (int(row[0])*60)+int(row[1]))

In [17]:
adj['D/N'] = adj['D/N'].map({'N':0, 'D':1})

In [18]:
adj['Rescheduled'] = adj[pd.notnull(adj['W/L'])]['Orig. Scheduled'].apply(lambda row: 0 if pd.isnull(row) == True else 1)
adj['Reschedule_Reason'] = adj[adj['Rescheduled'] == 1]['Orig. Scheduled'].apply(lambda row: row[row.find('(')+1:-1])
adj['Orig. Scheduled'] = adj[adj['Rescheduled'] == 1]['Orig. Scheduled'].apply(lambda row: row.split(' (')[0]).astype('datetime64')

In [ ]:
# Create "Interleague" variable        
AL = ["BAL","BOS","NYY","TBR","TOR",
      "CHW","CLE","DET","KCR","MIN",
      "HOU","LAA","OAK","SEA","TEX"] # Create list of AL teams
NL = ["ATL","MIA","NYM","PHI","WSN",
      "CHC","CIN","MIL","PIT","STL",
      "ARI","COL","LAD","SDP","SFG"] # Create list of NL teams

adj["Interleague"] = "" # Create empty "Interleague" column in adjframe

# Iterate through each row in adjframe to test if team and opponent are in same league
# If TRUE then Interleague = 0. If FALSE then Interleague = 1
for i in range(0,len(adj)):
    if (adj.iloc[i,1] in AL and adj.iloc[i,3] in AL) or \
       (adj.iloc[i,1] in NL and adj.iloc[i,3] in NL):
        adj.iloc[i,-1] = 0
    else:
        adj.iloc[i,-1] = 1
        
# Create "Rival" variable
ALE = ["BAL","BOS","NYY","TBR","TOR"] # Create list of ALE teams
ALC = ["CHW","CLE","DET","KCR","MIN"] # Create list of ALC teams
ALW = ["HOU","LAA","OAK","SEA","TEX"] # Create list of ALW teams
NLE = ["ATL","MIA","NYM","PHI","WSN"] # Create list of NLE teams
NLC = ["CHC","CIN","MIL","PIT","STL"] # Create list of NLC teams
NLW = ["ARI","COL","LAD","SDP","SFG"] # Create list of NLW teams

adj["Rival"] = "" # Create empty "Rival" column in adjframe

# Iterate through each row in adjframe to test if team and opponent are in same division
# If TRUE then Rival = 1. If FALSE then Rival = 0
for i in range(0,len(adj)):
    if (adj.iloc[i,1] in ALE and adj.iloc[i,3] in ALE) or \
       (adj.iloc[i,1] in ALC and adj.iloc[i,3] in ALC) or \
       (adj.iloc[i,1] in ALW and adj.iloc[i,3] in ALW) or \
       (adj.iloc[i,1] in NLE and adj.iloc[i,3] in NLE) or \
       (adj.iloc[i,1] in NLC and adj.iloc[i,3] in NLC) or \
       (adj.iloc[i,1] in NLW and adj.iloc[i,3] in NLW):
        adj.iloc[i,-1] = 1
    else:
        adj.iloc[i,-1] = 0

In [ ]:
adj['Gm#'] = adj.groupby(['Year','Tm']).cumcount() + 1
adj['HmGm#'] = adj.groupby(['Year','Tm','Home_Away']).cumcount() + 1
for i in range(0,len(adj)):
    if adj.iloc[i,2] != 1:
        adj.iloc[i,-1] = ''

In [ ]:
# adj.to_csv('MLBGameStats.csv', index=False)

In [ ]:
# import saspy

In [ ]:
# mysas = saspy.SASsession(cfgname='iomwin')

In [ ]:
# mysas.df2sd(adj,table='MLB_GAME_STATS',libref="lcssan")